# Sheet Flow
- import functions
- define track of interest based on sheet_gid dict
- read in weekend dataframe
- run function drop_empties
- run function check_df
- produce these dfs and dicts:
    - constructor_pricing, driver_pricing
- run function `score_race_qualifying_sprint_predicted`
    - function scores the entire weekend for drivers and constructors based on the predicted finishing places for qualifying and the race, and same for sprint if there's a sprint race
- look at current team composition, current team value, and mess with other combinations
- run function `main`
    - function evaluates all possible team combinations that i can afford, and then prints the top 100 based on maximum points value, taking into account the points penalty for extra (>2) trades
    - function does not evaluate fan-voted driver of the day, this is impossible to predict (+10 points)
    - function assumes winning driver sets the fastest lap (+10 points)
    - function assumes all gain/loss and overtakes happens on track and not as a result of how things shake out with the pit strategy

# Still To Do
- ~~make google sheet of teams and track team composition through each weekend~~
- experiment with analyses on this data

In [1]:
from weekend_functions import *

# Define Track and Read In Weekend Data

In [2]:
track_name = 'miami'

In [3]:
# read in from google sheets the sheet for this weekend's events
weekend_df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid={sheet_gid[track_name]}')

# read in from local sometimes needed when offline, make sure to download the file from sheets first...
# weekend_df = pd.read_csv(f'f1_main - {track_name}.csv')

weekend_df = drop_empties(weekend_df)
check_df(weekend_df)

weekend_df

,Team,Driver,predicted_sprint_qualifying_miami,predicted_sprint_race_miami,predicted_qualifying_miami,predicted_race_miami
0,Red Bull Racing-RBPT,Max Verstappen,1,1,1,1
1,Ferrari,Charles Leclerc,2,4,4,4
2,Red Bull Racing-RBPT,Sergio Perez,3,2,2,2
3,Ferrari,Carlos Sainz Jr.,4,5,5,5
4,McLaren-Mercedes,Lando Norris,5,3,3,3
5,McLaren-Mercedes,Oscar Piastri,7,6,7,6
6,Mercedes,Lewis Hamilton,8,8,9,7
7,Mercedes,George Russell,9,9,6,8
8,Aston Martin Aramco-Mercedes,Fernando Alonso,6,7,8,9
9,Aston Martin Aramco-Mercedes,Lance Stroll,10,10,10,10


In [4]:
weekend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Team                               20 non-null     object
 1   Driver                             20 non-null     object
 2   predicted_sprint_qualifying_miami  20 non-null     int64 
 3   predicted_sprint_race_miami        20 non-null     int64 
 4   predicted_qualifying_miami         20 non-null     int64 
 5   predicted_race_miami               20 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 1.1+ KB


# Driver and Constructor Prices

In [5]:
# pricing for drivers and constructors, updated each week with the latest values from the Formula 1 Fantasy Game
driver_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=920234107').dropna(axis=1, how='all')
constructor_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=1972297135').dropna(axis=1, how='all')

In [6]:
current_driver_values = {x[0]: x[1] for x in driver_pricing[['Driver', track_name]].values}
current_constructor_values = {x[0]: x[1] for x in constructor_pricing[['Constructor', track_name]].values}

In [7]:
current_driver_values

{'Max Verstappen': 30.1,
 'Lando Norris': 24.1,
 'Sergio Perez': 23.0,
 'Charles Leclerc': 22.0,
 'Carlos Sainz Jr.': 20.4,
 'Oscar Piastri': 19.9,
 'Lewis Hamilton': 19.0,
 'George Russell': 19.1,
 'Fernando Alonso': 16.5,
 'Lance Stroll': 13.0,
 'Daniel Ricciardo': 8.8,
 'Esteban Ocon': 9.0,
 'Yuki Tsunoda': 8.4,
 'Kevin Magnussen': 8.7,
 'Pierre Gasly': 7.8,
 'Alexander Albon': 7.5,
 'Nico Hulkenberg': 7.6,
 'Guanyu Zhou': 7.1,
 'Valtteri Bottas': 6.2,
 'Logan Sargeant': 6.3,
 'Oliver Bearman': nan}

In [8]:
current_constructor_values

{'Red Bull Racing-RBPT': 28.6,
 'McLaren-Mercedes': 24.1,
 'Mercedes': 20.2,
 'Ferrari': 21.5,
 'Aston Martin Aramco-Mercedes': 15.1,
 'VisaCashApp RB': 8.4,
 'Alpine-Renault': 8.4,
 'Stake F1 Team Kick Sauber': 5.9,
 'Williams-Mercedes': 6.3,
 'Haas-Ferrari': 8.2}

# Look at Predictions for the Weekend

In [9]:
drivers, constructors, driver_scores, constructor_scores, driver_score_summary, constructor_score_summary = score_race_qualifying_sprint_predicted(weekend_df, track_name)

Miami is a Sprint Race Weekend.


In [10]:
driver_scores

{'Max Verstappen': 58,
 'Charles Leclerc': 22,
 'Sergio Perez': 36,
 'Carlos Sainz Jr.': 19,
 'Lando Norris': 33,
 'Oscar Piastri': 19,
 'Lewis Hamilton': 13,
 'George Russell': 7,
 'Fernando Alonso': 5,
 'Lance Stroll': 2,
 'Daniel Ricciardo': -6,
 'Yuki Tsunoda': 2,
 'Valtteri Bottas': 8,
 'Nico Hulkenberg': 1,
 'Kevin Magnussen': 2,
 'Alexander Albon': 0,
 'Guanyu Zhou': 0,
 'Esteban Ocon': -3,
 'Pierre Gasly': 3,
 'Logan Sargeant': 6}

In [11]:
constructor_scores

{'Red Bull Racing-RBPT': 89,
 'Ferrari': 51,
 'McLaren-Mercedes': 62,
 'Mercedes': 30,
 'Aston Martin Aramco-Mercedes': 17,
 'VisaCashApp RB': -1,
 'Stake F1 Team Kick Sauber': 9,
 'Haas-Ferrari': 6,
 'Williams-Mercedes': 5,
 'Alpine-Renault': -1}

In [12]:
# F-winning current team

my_constructors = ['Ferrari', 'Red Bull Racing-RBPT']
my_drivers = ['Sergio Perez', 'Nico Hulkenberg', 'Daniel Ricciardo', 'Valtteri Bottas', 'Yuki Tsunoda']
remaining_cost_cap = 1.0

# my team predicted points and cost spent

team_score = sum(map(lambda x: driver_scores[x], my_drivers)) + max(map(lambda x: driver_scores[x], my_drivers)) + sum(map(lambda x: constructor_scores[x], my_constructors))
team_value = sum(map(lambda x: current_driver_values[x], my_drivers)) + sum(map(lambda x: current_constructor_values[x], my_constructors)) + remaining_cost_cap

print(f'Team score with {", ".join(my_drivers)} and {", ".join(my_constructors)}: {team_score}')
print(f'Team value (with remaining cost cap) as it is currently constructed: {round(team_value, 1)}')

Team score with Sergio Perez, Nico Hulkenberg, Daniel Ricciardo, Valtteri Bottas, Yuki Tsunoda and Ferrari, Red Bull Racing-RBPT: 217
Team value (with remaining cost cap) as it is currently constructed: 105.1


In [14]:
# this space is to mess aroud with other team combinations and look at the points and value, and remaining cost cap

potential_constructors = ['Red Bull Racing-RBPT', 'McLaren-Mercedes']
potential_drivers = ['Sergio Perez', 'Yuki Tsunoda', 'Valtteri Bottas', 'Nico Hulkenberg', 'Logan Sargeant']

potential_score = sum(map(lambda x: driver_scores[x], potential_drivers)) + max(map(lambda x: driver_scores[x], potential_drivers)) + sum(map(lambda x: constructor_scores[x], potential_constructors))
potential_cost = sum(map(lambda x: current_driver_values[x], potential_drivers)) + sum(map(lambda x: current_constructor_values[x], potential_constructors))
new_remaining_cost = team_value - potential_cost

print(f'potential score: {potential_score}')
print(f'potential cost: {potential_cost}')
print(f'new remaining cost: {round(new_remaining_cost, 2)}')

potential score: 240
potential cost: 104.2
new remaining cost: 0.9


# Evaulate Next Race

In [15]:
top_teams = main(my_drivers, my_constructors, weekend_df, track_name, driver_pricing, constructor_pricing, remaining_cost_cap)

Miami is a Sprint Race Weekend.
=== Predicted Driver Scores for Miami ===
{'Max Verstappen': 58, 'Charles Leclerc': 22, 'Sergio Perez': 36, 'Carlos Sainz Jr.': 19, 'Lando Norris': 33, 'Oscar Piastri': 19, 'Lewis Hamilton': 13, 'George Russell': 7, 'Fernando Alonso': 5, 'Lance Stroll': 2, 'Daniel Ricciardo': -6, 'Yuki Tsunoda': 2, 'Valtteri Bottas': 8, 'Nico Hulkenberg': 1, 'Kevin Magnussen': 2, 'Alexander Albon': 0, 'Guanyu Zhou': 0, 'Esteban Ocon': -3, 'Pierre Gasly': 3, 'Logan Sargeant': 6}

=== Current Team ===
Constructors: ['Ferrari', 'Red Bull Racing-RBPT']
Drivers: ['Sergio Perez', 'Nico Hulkenberg', 'Daniel Ricciardo', 'Valtteri Bottas', 'Yuki Tsunoda']
Current Team Value: 105.1
Current Available Value: 1.0
Total Number of Team Combinations: 697680
Total Number of Team Combinations I can afford: 425059
Explored all of the valid 425059 teams.

=== TEAM AT POSITION 1 WITH SCORE 240 ===
Constructor: ('Red Bull Racing-RBPT', 'McLaren-Mercedes')
Drivers: ('Sergio Perez', 'Yuki Tsuno